# Análisis de transacciones fraudulentas
##  20.468 movimientos con 114 características

El siguiente caso de estudio corresponde a **20.468 movimientos bancarios en los que existen transacciones fraudulentas** de modo que me dispongo a analizarlos utilizando **algoritmos de ciencia de datos para predecir si un movimiento bancario es fradulento o no**. El modelo entrenado que sea más eficiente a nivel de precisión sera el utilizado para futuros movimientos nuevos de este banco con estas características.

link data: https://www.kaggle.com/datasets/volodymyrgavrysh/fraud-detection-bank-dataset-20k-records-binary 

### Librerías

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

### Clase Algoritmos
Esta clase se encargará de calcular la precisión de los algoritmos siempre y cuando inicialice correctamente el constructor con sus parametros correctos. Después solo hay que llamar a la funcion **precision()**. También almacenará los datos de precisión en un dataframe **df** para una consulta posterior en el apartado de la conclusión.

In [2]:
datos = {'Nombre_Algoritmo':[], 'Precision':[]}  
df = pd.DataFrame(data=datos) 
class Algoritmos:
    
    def __init__(self,nombre,cm, y_test):
        self.nombre = nombre
        self.cm = cm
        self.y_test = y_test
        
    def __str__(self):
        return (self.nombre)
        
    def precision(self):
        global df
        assert(self.cm.shape==(2,2))
        assert(type(self.y_test) == pd.DataFrame)
        assert(type(self.cm[0][0]) == np.int64)
        precision_02 = ((self.cm[0][0] + self.cm[1][1])/len(self.y_test)) * 100
        assert(type(precision_02) == np.float64)
        df = df.append({'Nombre_Algoritmo':self.nombre, 'Precision':precision_02}, ignore_index=True)
        return("%.2f" %precision_02)

###  Carga y Visualización de datos

In [3]:
path = "C:/Users/USUARIO/Desktop/fraud_movents02/"
data = pd.read_csv(path + "fraud_detection_bank_dataset.csv")

In [4]:
data

,Unnamed: 0,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
0,0,9,1354,0,18,0,1,7,9,0,...,0,0,0,1,1,0,0,0,49,1
1,1,0,239,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,55,1
2,2,0,260,0,4,0,3,6,0,0,...,0,0,0,1,1,0,0,0,56,1
3,3,17,682,0,1,0,0,8,17,0,...,0,1,0,1,1,0,0,0,65,1
4,4,1,540,0,2,0,1,7,1,0,...,0,0,0,1,1,0,0,0,175,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20463,20463,0,88,0,0,0,2,-1,0,0,...,0,1,0,0,0,0,1,0,85,0
20464,20464,0,134,0,2,0,0,6,0,0,...,0,0,0,0,0,0,0,0,7,0
20465,20465,4,393,1,1,0,0,-1,4,0,...,0,1,0,0,0,0,0,0,45,0
20466,20466,0,10,0,1,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,5,0


In [5]:
# Visualizo cuantos movimimientos fraudulentos hay. 
p = data['targets'] == 1
filtro = data[p]
len(filtro.index)

5438

In [6]:
data.describe() 

,Unnamed: 0,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
count,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.0,...,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000
mean,10233.500000,3.226256,294.793043,0.420021,2.329343,0.083594,0.939857,2.386066,3.226256,0.0,...,0.004935,0.358120,0.002443,0.308384,0.190737,0.000049,0.049345,0.024282,43.486125,0.265683
std,5908.746991,20.564308,717.541984,7.367275,10.068512,0.840537,4.222896,3.018140,20.564308,0.0,...,0.073478,0.479459,0.049366,0.461837,0.392892,0.006990,0.216593,0.305079,59.217560,0.441707
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5116.750000,0.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
50%,10233.500000,0.000000,97.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.000000,0.000000
75%,15350.250000,2.000000,283.000000,0.000000,2.000000,0.000000,1.000000,6.000000,2.000000,0.0,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,61.250000,1.000000
max,20467.000000,2301.000000,37808.000000,904.000000,772.000000,54.000000,230.000000,11.000000,2301.000000,0.0,...,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,17.000000,747.000000,1.000000


### Preprocesado de datos

Primero compruebo si hay algun valor nulo en mi conjunto de datos. Luego observo el tipo de datos por si existiera alguna columna tipo object para transformarla. En este caso no es necesario ya que todas son entradas de datos tipo int64. 

In [7]:
# Preprocesado
data.duplicated().sum()  

0

In [8]:
# Preprocesado
data.isnull().sum()

Unnamed: 0    0
col_0         0
col_1         0
col_2         0
col_3         0
             ..
col_108       0
col_109       0
col_110       0
col_111       0
targets       0
Length: 114, dtype: int64

In [9]:
data.dtypes

Unnamed: 0    int64
col_0         int64
col_1         int64
col_2         int64
col_3         int64
              ...  
col_108       int64
col_109       int64
col_110       int64
col_111       int64
targets       int64
Length: 114, dtype: object

### Transformacíon de datos

Almaceno las columnas que considero variables independientes y dependiente, por ejemplo en este supuesto todas las columnas son importantes en el problema menos el indice del movimiento **'unnamed:0'**. El siguiente paso es considerar quien es la variable dependiente y las variables independientes:

- El objetivo de este supesto es predecir la columna **'targets'** para pronosticar si un movimiento es fraudulento  o no . De modo que la columna **'targets' será la variable dependiente**.

- Todas las columnas tienen relevancia en este problema entonces **las variables independientes seran todas las columnas menos 'targets' y 'unnamed:0'**

In [10]:
X = data.iloc[:, 1:-1]
Y = data.iloc[:, -1:]

In [11]:
X

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_102,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111
0,9,1354,0,18,0,1,7,9,0,0,...,4,0,0,0,1,1,0,0,0,49
1,0,239,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,55
2,0,260,0,4,0,3,6,0,0,0,...,1,0,0,0,1,1,0,0,0,56
3,17,682,0,1,0,0,8,17,0,0,...,1,0,1,0,1,1,0,0,0,65
4,1,540,0,2,0,1,7,1,0,0,...,1,0,0,0,1,1,0,0,0,175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20463,0,88,0,0,0,2,-1,0,0,0,...,0,0,1,0,0,0,0,1,0,85
20464,0,134,0,2,0,0,6,0,0,0,...,1,0,0,0,0,0,0,0,0,7
20465,4,393,1,1,0,0,-1,4,0,0,...,0,0,1,0,0,0,0,0,0,45
20466,0,10,0,1,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [12]:
Y

,targets
0,1
1,1
2,1
3,1
4,1
...,...
20463,0
20464,0
20465,0
20466,0


### TRAIN Y TEST
Después de visualizar los datos, preprocesarlos, considerar que columnas son relevantes para el caso de estudio y dividirlos en dos matrices ahora ya se pasaría a la fase de división de datos, es decir, considerar cuanto porcentaje de datos del dataset sería para entrenar el modelo, otro porcentaje para testearlo y verificar su precisión. Normalmente:

- **TRAIN**, tiene el **80% de datos del dataset original** ya que cuanto más datos más comprobaciones realizará y más fiable será.
- **TEST**, tiene el **20% de datos del dataset original** y con él se mide la precisión.

Como sería un proceso muy laborioso hacerlo manualmente con está gran cantidad de datos hay una función en Python del modulo **sklearn** llamada **train_test_split** que nos facilita esta división de datos.

In [13]:
# Divido los datos usando la funcion train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=(0))

# Escalado de variables
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# test con 4094 datos
# train con 16.374 datos

In [14]:
p = y_test['targets'] == 1
filtro = y_test[p]
len(filtro.index) # Tiene 1061 movimientos fraudulentos en el test de 4094

1061

### Algoritmo Regresión Logística
La regresión logística es un tipo de análisis de regresión utilizado para predecir el resultado de una variable categórica (una variable que puede adoptar un número limitado de categorías) en función de las variables independientes o predictoras.

In [15]:
LogisticR = LogisticRegression(random_state=0) # random state = 0
LogisticR.fit(X_train, y_train) #Ajustamos el modelo

# Realizo la predicción de los resultados con el conjunto de testing
y_pred = LogisticR.predict(X_test)

# Matriz de confusion
# De los 56.926 datos  entradas de clientes
cm = confusion_matrix(y_test, y_pred) # Precisión 88.40%
cm

array([[2952,   81],
       [ 394,  667]], dtype=int64)

In [16]:
cm[1][1]

667

In [17]:
Reg = Algoritmos("Regresión Logística", cm, y_test)
Reg.precision() 

'88.40'

### Análisis discriminante lineal
El análisis discriminante lineal (Linear Discriminant Analysis o LDA) es un método alternativo más adecuado a la regresión logística cuando la variable cualitativa tiene más de dos niveles (K ≥ 2). Supone también un modelo más estable cuando el tamaño muestral n es pequeño y la distribución de los predictores es aproximadamente normal en cada una de sus clases. El propósito del LDA es encontrar la combinación lineal de las variables originales que permita la mejor separación entre grupos de un set de datos.

In [18]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

y_pred = lda.predict(X_test)
cm = confusion_matrix(y_test, y_pred) # 85.91 % de precisión
cm

array([[2997,   36],
       [ 541,  520]], dtype=int64)

In [19]:
cm[1][1]

520

In [20]:
cl = Algoritmos("Análisis discriminante lineal", cm, y_test)
cl.precision()

'85.91'

### AdaBoost
Un clasificador AdaBoost es un metaestimador que comienza ajustando un clasificador en el conjunto de datos original y luego ajusta copias adicionales del clasificador en el mismo conjunto de datos, pero donde los pesos de las instancias clasificadas incorrectamente se ajustan de modo que los clasificadores posteriores se centren más en casos difícile

In [21]:
ad = AdaBoostClassifier(random_state=0)
ad.fit(X_train, y_train)

y_pred = ad.predict(X_test)

cm = confusion_matrix(y_test, y_pred) #  92.21%  de precision
cm

array([[2904,  129],
       [ 190,  871]], dtype=int64)

In [22]:
cm[1][1]

871

In [23]:
ada = Algoritmos("Análisis discriminante lineal", cm, y_test)
ada.precision()

'92.21'

### Algoritmo KNN (K Vecinos más cercanos)

Este algoritmo es un clasificador de aprendizaje supervisado no paramétrico, que utiliza la proximidad para hacer clasificaciones o predicciones sobre la agrupación de un punto de datos individual. Si bien se puede usar para problemas de regresión o clasificación, generalmente se usa como un algoritmo de clasificación, partiendo de la suposición de que se pueden encontrar puntos similares cerca uno del otro.

In [24]:
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2) # Objeto clasifier
KNN.fit(X_train, y_train)

y_pred = KNN.predict(X_test)
cm = confusion_matrix(y_test, y_pred) #  90.16% de precision
cm

array([[2868,  165],
       [ 238,  823]], dtype=int64)

In [25]:
cm[1][1]

823

In [26]:
KNN = Algoritmos("KNV", cm, y_test) 
KNN.precision()

'90.16'

### Algoritmo Teorema de Bayes
El teorema de Bayes es utilizado para calcular la probabilidad de un suceso, teniendo información de antemano sobre ese suceso. Podemos calcular la probabilidad de un suceso A, sabiendo además que ese A cumple cierta característica que condiciona su probabilidad. El teorema de Bayes entiende la probabilidad de forma inversa al teorema de la probabilidad total. El teorema de la probabilidad total hace inferencia sobre un suceso B, a partir de los resultados de los sucesos A. Por su parte, Bayes calcula la probabilidad de A condicionado a B.

In [27]:
BAYES = GaussianNB()
BAYES.fit(X_train, y_train)

y_pred = BAYES.predict(X_test)
cm = confusion_matrix(y_test, y_pred) #83.98% de precision
cm

array([[2882,  151],
       [ 505,  556]], dtype=int64)

In [28]:
cm[1][1]

556

In [29]:
TeorBayes = Algoritmos("Teorema de Bayes", cm, y_test)
TeorBayes.precision()

'83.98'

### Árbol de Clasificación
Un árbol de clasificación es un tipo de árbol de decisiones. Utiliza la medida de impurezas de Gini para clasificar los registros en las categorías del campo objetivo. Las predicciones se basan en combinaciones de valores en los campos de entrada.
Un árbol de clasificación calcula la categoría de destino pronosticada para cada nodo en el árbol.

In [30]:
DecisionTree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
DecisionTree.fit(X_train, y_train)

y_pred = DecisionTree.predict(X_test)
cm = confusion_matrix(y_test, y_pred)  # 89.59% de precision
cm

array([[2807,  226],
       [ 200,  861]], dtype=int64)

In [31]:
cm[1][1]

861

In [32]:
Arbol = Algoritmos("Árboles de Clasificación", cm, y_test)
Arbol.precision()

'89.59'

In [33]:
# Aleatorio Entropia
rand = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rand.fit(X_train, y_train)

y_pred = rand.predict(X_test)
cm = confusion_matrix(y_test, y_pred) # 92.26% precision
cm

array([[2946,   87],
       [ 230,  831]], dtype=int64)

In [34]:
cm[1][1]

831

In [35]:
ArbolRandomEntropia = Algoritmos("Árboles Aleatorios Entropia", cm, y_test)
ArbolRandomEntropia.precision()

'92.26'

In [36]:
# Aleatorio Gini
rand02 = RandomForestClassifier(n_estimators = 10, criterion = 'gini', random_state = 0)
rand02.fit(X_train, y_train)

y_pred = rand02.predict(X_test)
cm = confusion_matrix(y_test, y_pred) # 92.26% precision
cm

array([[2948,   85],
       [ 235,  826]], dtype=int64)

In [37]:
cm[1][1]

826

In [38]:
ArbolRandomGini = Algoritmos("Árboles Aleatorios Gini", cm, y_test)
ArbolRandomGini.precision()

'92.18'

### Algoritmo de XGBOOST
XGBoost es una biblioteca de aumento de gradiente distribuida optimizada diseñada para ser altamente eficiente , flexible y portátil . Implementa algoritmos de aprendizaje automático bajo el marco Gradient Boosting . XGBoost proporciona un impulso de árbol paralelo (también conocido como GBDT, GBM) que resuelve muchos problemas de ciencia de datos de una manera rápida y precisa

In [39]:
XGBOOST = XGBClassifier()
XGBOOST.fit(X_train, y_train)

y_pred = XGBOOST.predict(X_test)
cm = confusion_matrix(y_test, y_pred) #  93.99 % precision
cm

array([[2933,  100],
       [ 146,  915]], dtype=int64)

In [40]:
cm[1][1]

915

In [41]:
xgb = Algoritmos("XGBOOST", cm, y_test)
xgb.precision()

'93.99'

### Algoritmo Red Neuronal Artificial
Una red neuronal es un modelo simplificado que emula el modo en que el cerebro humano procesa la información: Funciona simultaneando un número elevado de unidades de procesamiento interconectadas que parecen versiones abstractas de neuronas.
La red aprende examinando los registros individuales, generando una predicción para cada registro y realizando ajustes a las ponderaciones cuando realiza una predicción incorrecta. Este proceso se repite muchas veces y la red sigue mejorando sus predicciones hasta haber alcanzado uno o varios criterios de parada.

In [43]:
# red neuronal artificial con keras del modelo secuencial
diccionario = {0:'Adadelta',1:'Adagrad',2:'Adam',3:'Adamax',4:'Ftrl',5:'Nadam',6:'RMSprop',7:'SGD'}
for optimi in diccionario.values():
    RNA = tf.keras.models.Sequential()
    # 1Âº Capa de inputs
    RNA.add(tf.keras.layers.Dense(units=114, activation='relu'))
    # 2Âº Capa oculta
    RNA.add(tf.keras.layers.Dense(units=50, activation='relu'))
    # Capa oculta
    RNA.add(tf.keras.layers.Dense(units=25, activation='relu'))
    #  Capa Capa oculta
    RNA.add(tf.keras.layers.Dense(units=10, activation='relu'))
    # Ultima capa de salidad, con funcion sigmoide
    RNA.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    RNA.compile(optimizer = optimi, loss = 'binary_crossentropy', metrics = ['accuracy'])
    RNA.fit(X_train, y_train, batch_size = 64, epochs = 200)
    
    y_pred = RNA.predict(X_test)
    y_pred_02 = (y_pred > 0.5)
    y_pred_02 = np.where(y_pred_02=='True',1,y_pred_02)
    cm = confusion_matrix(y_test, y_pred_02)    
    redneuro = Algoritmos("Red Neuronal Artificial Optimizador: " + str(optimi), cm, y_test)
    redneuro.precision()

Epoch 1/200
256/256 [==============================] - 3s 4ms/step - loss: 0.7240 - accuracy: 0.3904
Epoch 2/200
256/256 [==============================] - 1s 5ms/step - loss: 0.7170 - accuracy: 0.4282
Epoch 3/200
256/256 [==============================] - 1s 4ms/step - loss: 0.7102 - accuracy: 0.4650
Epoch 4/200
256/256 [==============================] - 1s 4ms/step - loss: 0.7035 - accuracy: 0.5005
Epoch 5/200
256/256 [==============================] - 1s 5ms/step - loss: 0.6970 - accuracy: 0.5399
Epoch 6/200
256/256 [==============================] - 1s 4ms/step - loss: 0.6906 - accuracy: 0.5806
Epoch 7/200
256/256 [==============================] - 1s 4ms/step - loss: 0.6843 - accuracy: 0.6147
Epoch 8/200
256/256 [==============================] - 1s 3ms/step - loss: 0.6781 - accuracy: 0.6449
Epoch 9/200
256/256 [==============================] - 1s 3ms/step - loss: 0.6719 - accuracy: 0.6718
Epoch 10/200
256/256 [==============================] - 1s 4ms/step - loss: 0.6658 - accura

256/256 [==============================] - 1s 3ms/step - loss: 0.4339 - accuracy: 0.8372
Epoch 82/200
256/256 [==============================] - 1s 3ms/step - loss: 0.4323 - accuracy: 0.8376
Epoch 83/200
256/256 [==============================] - 1s 3ms/step - loss: 0.4307 - accuracy: 0.8382
Epoch 84/200
256/256 [==============================] - 1s 3ms/step - loss: 0.4291 - accuracy: 0.8388
Epoch 85/200
256/256 [==============================] - 1s 3ms/step - loss: 0.4276 - accuracy: 0.8394
Epoch 86/200
256/256 [==============================] - 1s 3ms/step - loss: 0.4260 - accuracy: 0.8393
Epoch 87/200
256/256 [==============================] - 1s 3ms/step - loss: 0.4246 - accuracy: 0.8396
Epoch 88/200
256/256 [==============================] - 1s 4ms/step - loss: 0.4231 - accuracy: 0.8398
Epoch 89/200
256/256 [==============================] - 1s 4ms/step - loss: 0.4216 - accuracy: 0.8402
Epoch 90/200
256/256 [==============================] - 1s 3ms/step - loss: 0.4202 - accuracy: 

256/256 [==============================] - 1s 3ms/step - loss: 0.3545 - accuracy: 0.8581
Epoch 161/200
256/256 [==============================] - 1s 3ms/step - loss: 0.3538 - accuracy: 0.8581
Epoch 162/200
256/256 [==============================] - 1s 4ms/step - loss: 0.3532 - accuracy: 0.8580
Epoch 163/200
256/256 [==============================] - 1s 3ms/step - loss: 0.3526 - accuracy: 0.8581
Epoch 164/200
256/256 [==============================] - 1s 4ms/step - loss: 0.3520 - accuracy: 0.8581
Epoch 165/200
256/256 [==============================] - 1s 4ms/step - loss: 0.3514 - accuracy: 0.8581
Epoch 166/200
256/256 [==============================] - 1s 4ms/step - loss: 0.3509 - accuracy: 0.8580
Epoch 167/200
256/256 [==============================] - 1s 3ms/step - loss: 0.3503 - accuracy: 0.8581
Epoch 168/200
256/256 [==============================] - 1s 3ms/step - loss: 0.3498 - accuracy: 0.8582
Epoch 169/200
256/256 [==============================] - 1s 3ms/step - loss: 0.3492 - a

256/256 [==============================] - 1s 3ms/step - loss: 0.2545 - accuracy: 0.8994
Epoch 41/200
256/256 [==============================] - 1s 3ms/step - loss: 0.2532 - accuracy: 0.9004
Epoch 42/200
256/256 [==============================] - 1s 4ms/step - loss: 0.2520 - accuracy: 0.9008
Epoch 43/200
256/256 [==============================] - 1s 3ms/step - loss: 0.2509 - accuracy: 0.9014
Epoch 44/200
256/256 [==============================] - 1s 3ms/step - loss: 0.2497 - accuracy: 0.9014
Epoch 45/200
256/256 [==============================] - 1s 3ms/step - loss: 0.2486 - accuracy: 0.9022
Epoch 46/200
256/256 [==============================] - 1s 3ms/step - loss: 0.2475 - accuracy: 0.9028
Epoch 47/200
256/256 [==============================] - 1s 3ms/step - loss: 0.2464 - accuracy: 0.9031
Epoch 48/200
256/256 [==============================] - 1s 3ms/step - loss: 0.2454 - accuracy: 0.9033
Epoch 49/200
256/256 [==============================] - 1s 3ms/step - loss: 0.2443 - accuracy: 

256/256 [==============================] - 1s 3ms/step - loss: 0.1998 - accuracy: 0.9224
Epoch 121/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1994 - accuracy: 0.9228
Epoch 122/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1991 - accuracy: 0.9224
Epoch 123/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1987 - accuracy: 0.9223
Epoch 124/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1983 - accuracy: 0.9228
Epoch 125/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1980 - accuracy: 0.9227
Epoch 126/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1976 - accuracy: 0.9230
Epoch 127/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1972 - accuracy: 0.9238
Epoch 128/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1969 - accuracy: 0.9233
Epoch 129/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1965 - a

...

256/256 [==============================] - 1s 4ms/step - loss: 0.0705 - accuracy: 0.9746
Epoch 76/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0761 - accuracy: 0.9722
Epoch 77/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0720 - accuracy: 0.9744
Epoch 78/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0787 - accuracy: 0.9730
Epoch 79/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0648 - accuracy: 0.9745
Epoch 80/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0693 - accuracy: 0.9738
Epoch 81/200
256/256 [==============================] - 1s 5ms/step - loss: 0.0780 - accuracy: 0.9754
Epoch 82/200
256/256 [==============================] - 1s 4ms/step - loss: 0.0647 - accuracy: 0.9753
Epoch 83/200
256/256 [==============================] - 1s 4ms/step - loss: 0.0749 - accuracy: 0.9751
Epoch 84/200
256/256 [==============================] - 1s 4ms/step - loss: 0.0646 - accuracy: 

256/256 [==============================] - 1s 4ms/step - loss: 0.0550 - accuracy: 0.9806
Epoch 155/200
256/256 [==============================] - 1s 4ms/step - loss: 0.0554 - accuracy: 0.9806
Epoch 156/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0648 - accuracy: 0.9799
Epoch 157/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0706 - accuracy: 0.9811
Epoch 158/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0548 - accuracy: 0.9796
Epoch 159/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0515 - accuracy: 0.9809
Epoch 160/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0752 - accuracy: 0.9795
Epoch 161/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0535 - accuracy: 0.9801
Epoch 162/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0600 - accuracy: 0.9805
Epoch 163/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0580 - a

256/256 [==============================] - 1s 3ms/step - loss: 0.1564 - accuracy: 0.9389
Epoch 35/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1550 - accuracy: 0.9398
Epoch 36/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1529 - accuracy: 0.9400
Epoch 37/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1522 - accuracy: 0.9401
Epoch 38/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1522 - accuracy: 0.9414
Epoch 39/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1505 - accuracy: 0.9404
Epoch 40/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1508 - accuracy: 0.9409
Epoch 41/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1483 - accuracy: 0.9417
Epoch 42/200
256/256 [==============================] - 1s 3ms/step - loss: 0.1494 - accuracy: 0.9422
Epoch 43/200
256/256 [==============================] - 1s 4ms/step - loss: 0.1451 - accuracy: 

256/256 [==============================] - 1s 5ms/step - loss: 0.0998 - accuracy: 0.9634
Epoch 115/200
256/256 [==============================] - 1s 4ms/step - loss: 0.0996 - accuracy: 0.9628
Epoch 116/200
256/256 [==============================] - 1s 4ms/step - loss: 0.1042 - accuracy: 0.9631
Epoch 117/200
256/256 [==============================] - 1s 5ms/step - loss: 0.1021 - accuracy: 0.9628
Epoch 118/200
256/256 [==============================] - 1s 5ms/step - loss: 0.0973 - accuracy: 0.9639
Epoch 119/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0934 - accuracy: 0.9638
Epoch 120/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0944 - accuracy: 0.9638
Epoch 121/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0924 - accuracy: 0.9651
Epoch 122/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0925 - accuracy: 0.9639
Epoch 123/200
256/256 [==============================] - 1s 4ms/step - loss: 0.0920 - a

256/256 [==============================] - 1s 5ms/step - loss: 0.0755 - accuracy: 0.9692
Epoch 194/200
256/256 [==============================] - 1s 5ms/step - loss: 0.0750 - accuracy: 0.9693
Epoch 195/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0736 - accuracy: 0.9706
Epoch 196/200
256/256 [==============================] - 1s 6ms/step - loss: 0.0726 - accuracy: 0.9710
Epoch 197/200
256/256 [==============================] - 1s 5ms/step - loss: 0.0744 - accuracy: 0.9704
Epoch 198/200
256/256 [==============================] - 1s 4ms/step - loss: 0.0742 - accuracy: 0.9703
Epoch 199/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0716 - accuracy: 0.9715
Epoch 200/200
256/256 [==============================] - 1s 3ms/step - loss: 0.0742 - accuracy: 0.9711


### Conclusión

Después de testear y ajustar los datos con **16 modelos distintos de algoritmos**, **la mejor precisión** la realiza el algoritmo de **Xgboost** con un **93.99 %**.

Si observo su matriz de confusión del testing se pueder ver que:
- ([[2933,  100],[ 146,  915]], de los **4094 movimientos ha acertado correctamente 915 transacciones fraudulentas y 2933 transacciones no fraudulentas**.

Este modelo sería muy eficaz para pronosticar cualquier otra transaccion fraudulenta con estas mismas características en este banco.

In [49]:
df.sort_values(by='Precision',ascending=False)

,Nombre_Algoritmo,Precision
8,XGBOOST,93.991207
6,Árboles Aleatorios Entropia,92.256961
2,Análisis discriminante lineal,92.208109
7,Árboles Aleatorios Gini,92.183683
10,Red Neuronal Artificial Optimizador: Adagrad,91.744016
12,Red Neuronal Artificial Optimizador: Adamax,91.157792
14,Red Neuronal Artificial Optimizador: Nadam,91.108940
11,Red Neuronal Artificial Optimizador: Adam,90.962384
15,Red Neuronal Artificial Optimizador: RMSprop,90.522716
3,KNV,90.156326


Si quisiera saber que **importancia le ha dado a las columnas/características** haciendo las clasificaciones para determinar si un transacción es fraudulenta o no, a continuación lo muestro:

In [52]:
# Voy a ver que columnas le ha dado mas importancia para el modelo
col_imp = XGBOOST.feature_importances_
col_name = XGBOOST.get_booster().feature_names
dataXgboost = pd.DataFrame(col_imp, col_name)
dataXgboost = dataXgboost.rename(columns={0:'Importancia %'})
orden_desce = dataXgboost.sort_values('Importancia %',ascending=False)
orden_desce = orden_desce.iloc[:] * 100
print(orden_desce.head(10))

     Importancia %
83       23.584557
5        12.259048
106      11.187807
54        4.734734
74        2.916551
4         2.846203
26        2.453791
102       2.006691
76        1.680843
95        1.438495


En resumen, el algoritmo **XGBOOST con una precisión de casi 94%** ha tomado con mayor peso estadístico para predecir si un movimiento es fraudulento o no las columnas/características:

|**Columna**|**Importancia %**|
|------|------|
|83  |23.584557  |
|5  |12.259048  |
|106  |11.187807  |
|54  |4.734734  |
|74  |2.916551  |
|4  |2.846203  |
|26  |2.453791  |
|102  |2.006691  |
|76  |1.680843  |
|95  |1.438495  |


**Siendo las características 83, 5 y 106 muy determinantes.** Esto es muy útil si queremos que el banco realice a priori ciertos "controles/filtros" a sus clientes respecto a estas características, de modo que lo que podría ser realmente eficiente sería poner una alerta si el cliente realiza un movimiento y hay una alta desviación en estas 3 columnas junto con el modelo de Xgboost que te predice si una transacción es fraudulenta o no.


**Autor: Carlos Mir Martínez**

**Fecha: 22/06/2022**